# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
# !pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
# !pip install -qU ragas

As well, instead of the remote hosted solution that we used last week (Pinecone), we'll be leveraging Meta's [FAISS](https://github.com/facebookresearch/faiss) as the backend for our LangChain `VectorStore`.

We'll also install `unstructured` (from [Unstructured-IO](https://github.com/Unstructured-IO/unstructured)) and its dependencies which will allow us to load PDFs using the `UnstructuredPDFLoader` in the `langchain-community` package!

In [3]:
# !pip install -qU faiss_cpu pymupdf pandas
# !pip install python-dotenv

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass
from dotenv import load_dotenv
load_dotenv()

# openai.api_key = getpass("Please provide your OpenAI Key: ")
# os.environ["OPENAI_API_KEY"] = openai.api_key

True

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
!git clone https://github.com/AI-Maker-Space/DataRepository

Cloning into 'DataRepository'...


remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 54 (delta 15), reused 20 (delta 7), pack-reused 8
Receiving objects: 100% (54/54), 51.28 MiB | 45.94 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

documents = loader.load()

In [7]:
documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [9]:
len(documents)

159

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [11]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!

In [12]:
# 1. Approximate nearest neighbor
# 2. In memory
# 3. Tree division of vectors for logrithmic lookup
# 4. Compresses the vector to make it faster for lookup
# 5. Support both CPU/GPU

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [13]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [14]:
retrieved_documents = retriever.invoke("Who is the plantiff?")

In [15]:
for doc in retrieved_documents:
  print(doc)

page_content='would be owned by the foundation and used ‘for the good of the world’[.]” Plaintiff \nreplied: “Agree on all.” Ex. 2 at 1.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 27, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='property and derivative works funded by those monies, Plaintiff is presently unable to ascertain his \ninterest in or the use, allocation, or distribution of assets without an accounting. Plaintiff is therefore \nentitled to an accounting.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 32, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='1

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [48]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [49]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [50]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [51]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

Let's test it out!

In [53]:
question = "Who is the plantiff?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Elon Musk


In [54]:
question = "What does this complaint pertain to?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

The complaint pertains to breach of fiduciary duty, unfair business practices, and accounting.
[Document(page_content='1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 31 – \nCOMPLAINT \n \nTHIRD CAUSE OF ACTION \nBreach of Fiduciary Duty  \nAgainst All Defendants \n133. \nPlaintiff realleges and incorporates by reference only paragraphs of this Complaint \nnecessary for his claim of Breach of Fiduciary Duty. \n134. \nUnder California law, Defendants owe fiduciary duties to Plaintiff, including a duty \nto use Plaintiff’s contributions for the purposes for which they were made. E.g., Cal. Bus. & Prof. \nCode § 17510.8. Defendants have repeatedly breached their fiduciary duties to Plaintiff, including \nby:', metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 30, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [55]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

In [56]:
# 1. This is because we want to test how well our chunk size performs in our evaluation

In [57]:
len(documents)

159

In [58]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()
testset = generator.generate_with_langchain_docs(
    documents,
    test_size=10,
    distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5},
)

Filename and doc_id are the same for all nodes.                   
Generating: 100%|██████████| 10/10 [02:29<00:00, 14.93s/it]


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

In [59]:
# 50% simple questions, 25% reasoning questions, 25% multi_context questions

Let's look at the output and see what we can learn about it!

In [128]:
import numpy as np
idx = np.random.choice(len(testset.test_data))
print('idx: ', idx)
print(testset.test_data[idx].question)
print(testset.test_data[idx].ground_truth)
print(len(testset.test_data[idx].contexts))
print(testset.test_data[idx].contexts)

idx:  6
What worried Mr. Musk about Google's acquisition of DeepMind's AI technology and its impact on humans and AI regulation?
Mr. Musk was deeply concerned that DeepMind’s AI technology would be in the hands of someone who viewed it and its power so cavalierly, and could hide its design and capabilities behind closed doors.
1
['1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 10 – \nCOMPLAINT \n \nhe favored the human species over intelligent machines. Mr. Musk responded, “Well, yes, I am pro-\nhuman.” \n37. \nAt the end of 2013, Mr. Musk learned to his grave concern that Google was planning \nto acquire DeepMind. At the time, DeepMind was one of the most advanced AI companies in the \nindustry. Thus, Mr. Musk was deeply concerned that DeepMind’s AI technology would be in the \nhands of someone who viewed it and its power so cavalierly, and could hide its design and \ncapabilities behind closed

In [129]:
print(testset.test_data[0])

question="What is OpenAI's commitment to ensuring the benefit of all in their mission?" contexts=['feedback. The draft charter described OpenAI’s mission as to ensure that AGI “benefits all of \nhumanity.” It stated, “We commit to use any influence we obtain over AGI’s deployment to ensure \nit is used for the benefit of all, and to avoid enabling uses of AI or AGI that harm humanity or unduly \nconcentrate power. Our primary fiduciary duty is to humanity. We anticipate needing to marshal \nsubstantial resources to fulfill our mission, but will always assiduously act to minimize conflicts of \ninterest . . . that could compromise broad benefit.” \n67. \nOn March 11, 2019, OpenAI, Inc. announced that it would be creating a for-profit'] ground_truth="OpenAI's commitment is to use any influence they obtain over AGI's deployment to ensure it is used for the benefit of all and to avoid enabling uses of AI or AGI that harm humanity or unduly concentrate power." evolution_type='simple'


### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [130]:
test_df = testset.to_pandas()

In [131]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,What is OpenAI's commitment to ensuring the be...,[feedback. The draft charter described OpenAI’...,OpenAI's commitment is to use any influence th...,simple,True
1,What is the concept of Artificial General Inte...,[food is shown in a photo. One of the hallmark...,The concept of Artificial General Intelligence...,simple,True
2,What were the benefits of releasing models pub...,"[challenging.” At the time, OpenAI stated that...",nan,reasoning,True
3,What caused Mr. Musk to be deeply troubled abo...,[a superhuman level of play in the games of ch...,DeepMind's partnership with Google caused Mr. ...,reasoning,True
4,"What was OpenAI, Inc.'s initial research appro...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,"OpenAI, Inc.'s initial research approach was p...",multi_context,True
5,What was GPT-4's score on the Advanced Sommeli...,[The 2023 Breach Of The Founding Agreement \n2...,GPT-4 scored a 77% on the Advanced Sommelier e...,multi_context,True
6,What worried Mr. Musk about Google's acquisiti...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,Mr. Musk was deeply concerned that DeepMind’s ...,multi_context,True
7,"""What was the impact of OpenAI's models on fut...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,Their publication did prove to be useful to th...,multi_context,True
8,"""What's the name of OpenAI's second-gen model ...",[which needed to be trained on a specific task...,The name of OpenAI's second-gen model that exc...,multi_context,True
9,What percentile did GPT-4 score on the GRE Ver...,[The 2023 Breach Of The Founding Agreement \n2...,GPT-4 scored in the 99th percentile on the GRE...,simple,True


In [132]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [133]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Task was destroyed but it is pending!
task: <Task pending name='Task-1770' coro=<as_completed.<locals>.sema_coro() done, defined at /home/william.wetzel/venvs/w4d1/lib/python3.10/site-packages/ragas/executor.py:34> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /usr/lib/python3.10/asyncio/futures.py:385, Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /usr/lib/python3.10/asyncio/tasks.py:558]>
Task was destroyed but it is pending!
task: <Task pending name='Task-1771' coro=<as_completed.<locals>.sema_coro() done, defined at /home/william.wetzel/venvs/w4d1/lib/python3.10/site-packages/ragas/executor.py:34> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /usr/lib/python3.10/asyncio/futures.py:385, Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /usr/lib/python3.10/asyncio/tasks.py:558]>
Task was destroyed but it is pending!
task: <Task pending name='Task-1778' coro=<as_completed.<locals>.sema_co

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [98]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [135]:
response_dataset[2]

{'question': 'What were the benefits of releasing models publicly according to the paper co-authored by OpenAI scientists and independent social and technical scientists?',
 'answer': 'The benefits of releasing models publicly according to the paper co-authored by OpenAI scientists and independent social and technical scientists included the enhancement and extension of models by entire communities, spreading to open-source efforts and commercial entities.',
 'contexts': ['challenging.” At the time, OpenAI stated that it was releasing the full, open version with the hope \nthat it “will be useful to developers of future powerful models.” This release was accompanied by a \ndetailed paper co-authored by OpenAI scientists as well as independent social and technical \nscientists. This paper explained just some of the many benefits that came from releasing models \npublically as opposed to keeping them closed.',
  '1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [100]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [101]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 50/50 [00:28<00:00,  1.73it/s]


In [102]:
results

{'faithfulness': 0.9000, 'answer_relevancy': 0.9533, 'context_recall': 0.9500, 'context_precision': 0.8333, 'answer_correctness': 0.7075}

In [103]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is OpenAI's commitment to ensuring the be...,"OpenAI's commitment is to ensure that AGI ""ben...",[a key role in recruiting world-class talent t...,OpenAI's commitment is to use any influence th...,1.0,0.925887,1.0,0.750000,0.619513
1,What is the concept of Artificial General Inte...,The concept of Artificial General Intelligence...,[food is shown in a photo. One of the hallmark...,The concept of Artificial General Intelligence...,1.0,1.000000,1.0,0.833333,0.623467
2,What were the benefits of releasing models pub...,The benefits of releasing models publicly acco...,"[challenging.” At the time, OpenAI stated that...",nan,1.0,0.978097,1.0,0.000000,0.182732
3,What caused Mr. Musk to be deeply troubled abo...,Google,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,DeepMind's partnership with Google caused Mr. ...,0.0,0.932400,1.0,1.000000,0.951667
4,"What was OpenAI, Inc.'s initial research appro...","OpenAI, Inc.'s initial research approach was t...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,"OpenAI, Inc.'s initial research approach was p...",1.0,0.946466,1.0,1.000000,0.734327
5,What was GPT-4's score on the Advanced Sommeli...,GPT-4 scored a 77% on the Advanced Sommelier e...,[dramatically compressing. \n86. \nOn March 14...,GPT-4 scored a 77% on the Advanced Sommelier e...,1.0,0.960305,0.5,1.000000,0.540668
6,What worried Mr. Musk about Google's acquisiti...,Mr. Musk was worried that Google's acquisition...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,Mr. Musk was deeply concerned that DeepMind’s ...,1.0,0.965666,1.0,1.000000,0.612497
7,"""What was the impact of OpenAI's models on fut...",The impact of OpenAI's models on future model ...,"[challenging.” At the time, OpenAI stated that...",Their publication did prove to be useful to th...,1.0,0.941020,1.0,1.000000,0.839459
8,"""What's the name of OpenAI's second-gen model ...",GPT-2,[which needed to be trained on a specific task...,The name of OpenAI's second-gen model that exc...,1.0,0.883030,1.0,0.750000,0.970467
9,What percentile did GPT-4 score on the GRE Ver...,GPT-4 scored in the 99th percentile on the GRE...,[dramatically compressing. \n86. \nOn March 14...,GPT-4 scored in the 99th percentile on the GRE...,1.0,1.000000,1.0,1.000000,1.000000


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

In [104]:
from langchain.retrievers import MultiQueryRetriever

'''
The MultiQueryRetriever automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the MultiQueryRetriever might be able to overcome some of the limitations of the distance-based retrieval and get a richer set of results.
'''



advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [105]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [106]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [107]:
response = retrieval_chain.invoke({"input": "Who is the plantiff?"})
print(response["answer"])

In [111]:
response = retrieval_chain.invoke({"input": "How much money is at risk?"})
print(response["answer"])

The amount of money at risk is presently unknown and will be proven at trial, but it substantially exceeds the court's jurisdictional minimum of $35,000.


In [112]:
response = retrieval_chain.invoke({"input": "What does Elon Musk hope to gain from this lawsuit?"})
print(response["answer"])

Elon Musk is alleging breach of contract, promissory estoppel, breach of fiduciary duty, and unfair competition against the defendants. From the context provided, it seems that Musk hopes to hold the defendants accountable for their actions and potentially seek damages or other legal remedies related to these claims.


In [109]:
response = retrieval_chain.invoke({"input": "What does this complaint pertain to?"})
print(response["answer"])

Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [45]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [46]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [113]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating: 100%|██████████| 50/50 [04:11<00:00,  5.02s/it]


In [114]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What did researchers at the University of Toky...,Researchers at the University of Tokyo and Goo...,[implementation for others to build on. \n84. ...,Researchers at the University of Tokyo and Goo...,NaN,0.880486,0.500000,1.000000,0.743243
1,How did OpenAI use reinforcement learning in t...,OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,0.666667,0.872285,0.500000,1.000000,0.540233
2,"""What Google algorithm tackles challenges in d...",The Google algorithm that tackles challenges i...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The Transformer algorithm tackles challenges i...,1.000000,0.968951,1.000000,1.000000,0.740508
3,What game did OpenAI's initial work follow Dee...,OpenAI's initial work followed DeepMind's use ...,[77. \nInitial work at OpenAI followed much in...,Dota 2,1.000000,0.944019,1.000000,1.000000,0.961664
4,"""What was the impact of OpenAI's GPT-3 on futu...",The release of OpenAI's GPT-3 in 2020 had a si...,[which needed to be trained on a specific task...,Their publication did prove to be useful to th...,1.000000,0.926620,1.000000,0.679167,0.767369
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning to beat a w...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to beat a w...,NaN,0.933710,1.000000,0.700000,0.999786
6,What is the name of Mr. Musk's new lab announc...,The name of Mr. Musk's new lab announced on De...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,"OpenAI, Inc.",1.000000,0.956223,1.000000,0.142857,0.971514
7,What were Mr. Musk's concerns about AI and how...,Mr. Musk's concerns about AI were that artific...,"[Page, then-CEO of Google’s parent company Alp...",Mr. Musk's concerns about AI were that it coul...,1.000000,0.947023,1.000000,1.000000,0.618027
8,Who else expressed concerns about AGI before E...,Stephen Hawking and Sun Microsystems founder B...,[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking and Bill Joy expressed concern...,1.000000,0.963066,1.000000,1.000000,0.744882
9,What are some of the models released by OpenAI...,"OpenAI released models such as GPT-2, GPT-3, a...","[challenging.” At the time, OpenAI stated that...",OpenAI has released two models: GPT and GPT-2....,0.800000,0.940595,0.333333,0.250000,0.444804


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [115]:
results

{'faithfulness': 0.9000, 'answer_relevancy': 0.9533, 'context_recall': 0.9500, 'context_precision': 0.8333, 'answer_correctness': 0.7075}

And see how our advanced retrieval modified our chain!

In [116]:
advanced_retrieval_results

{'faithfulness': 0.9333, 'answer_relevancy': 0.9333, 'context_recall': 0.8333, 'context_precision': 0.7772, 'answer_correctness': 0.7532}

In [117]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.900000,0.933333,0.033333
1,answer_relevancy,0.953287,0.933298,-0.019989
2,context_recall,0.950000,0.833333,-0.116667
3,context_precision,0.833333,0.777202,-0.056131
4,answer_correctness,0.707480,0.753203,0.045723


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [118]:
# Instantiating the OpenAIEmbeddings model text-embedding-3-small
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [119]:
# Send the documents to the text-embedding-3-small model to get the new embeddings and load into FAISS index
vector_store = FAISS.from_documents(documents, new_embeddings)

In [120]:
# Get a retriever
new_retriever = vector_store.as_retriever()

In [121]:
# Create a new MultiQueryRetriever with the new retriever and the primary_qa_llm
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [122]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [123]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [124]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [125]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating: 100%|██████████| 50/50 [01:32<00:00,  1.85s/it]


In [126]:
new_advanced_retrieval_results

{'faithfulness': 0.9815, 'answer_relevancy': 0.9493, 'context_recall': 0.9500, 'context_precision': 0.8261, 'answer_correctness': 0.7830}

In [127]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.900000,0.933333,0.981481,0.048148,0.081481
1,answer_relevancy,0.953287,0.933298,0.949339,0.016042,-0.003948
2,context_recall,0.950000,0.833333,0.950000,0.116667,0.000000
3,context_precision,0.833333,0.777202,0.826111,0.048909,-0.007222
4,answer_correctness,0.707480,0.753203,0.783001,0.029798,0.075521


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

In [ ]:
# Yes the scores are higher across the board than ADA, and even beat the baseline on some records.

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [ ]:
### YOUR CODE HERE